In [1]:
import time as time
import threading
import copy

#define time variable
t = 0

# Define objects inside the game

#Furniture
couch = {"name": "couch", "type": "furniture",}
piano = {"name": "piano", "type": "furniture",}
queen_bed = {"name": "queen bed", "type": "furniture",}
double_bed = {"name": "double bed", "type": "furniture",}
dresser = {"name": "dresser", "type": "furniture",}
dinning_table = {"name": "dinning room", "type": "furniture",}

## new furniture
carpet = {"name": "carpet", "type": "furniture",}
table = {"name": "table", "type": "furniture",}
chair = {"name": "chair", "type": "furniture",}
lamp = {"name": "lamp", "type": "furniture",}
desk = {"name": "desk", "type": "furniture",}
picture = {"name": "picture", "type": "furniture",}
big_picture = {"name": "big picture", "type": "furniture",}
mirror = {"name": "mirror", "type": "furniture",}
sofa = {"name": "sofa", "type": "furniture",}
chest = {"name": "chest", "type": "furniture",}
tv = {"name": "tv", "type": "furniture",}

## Doors
door_1 = {"name": "door 1", "type": "door",}
door_2 = {"name": "door 2", "type": "door",}
door_3 = {"name": "door 3", "type": "door",}
door_4 = {"name": "door 4", "type": "door",}

#### Keys

key_a = {"name": "shinning key", "type": "key","target": door_1,}
key_b = {"name": "ancient key", "type": "key", "target": door_2,}
key_c = {"name": "slim key", "type": "key", "target": door_3,}
key_d = {"name": "long key", "type": "key", "target": door_4,}

## Rooms

game_room = {"name": "game room", "type": "room",}
bedroom_1 = {"name": "bedroom 1", "type": "room",}
bedroom_2 = {"name": "bedroom 2", "type": "room",}
living_room = {"name": "living room", "type": "room",}
outside = { "name": "outside"}

all_rooms = [game_room, bedroom_1, bedroom_2, living_room, outside]
all_doors = [door_1, door_2, door_3, door_4]

# define which items/rooms are related

object_relations = {    
    #first room
    "game room": [couch, piano, carpet, table, chair, lamp, door_1],
    "piano": [key_a],
    "door 1": [game_room, bedroom_1],
    #second room
    "bedroom 1": [door_1, carpet, queen_bed, picture, desk, door_2, door_3],
    "queen bed": [key_b],
    "door 2": [bedroom_1, bedroom_2],
    # third room
    "bedroom 2": [door_2, double_bed, dresser, carpet, chest, mirror],
    "double bed": [key_c],
    "dresser": [key_d],
    # 4th room
    "living room": [door_3, door_4, dinning_table, big_picture, sofa, tv],
    "door 3":[bedroom_1, living_room],
    "door 4":[living_room, outside],
    "outside": [door_4]    
}

#### other difficulties levels
object_relations_easy = {
    #first room
    "game room": [couch, piano, carpet, table, lamp, chair, door_1],
    "carpet": [key_a],
    "door 1": [game_room, outside],
    "outside": [door_1]
}

# define game state. Do not directly change this dict.
# Instead, when a new game starts, make a copy of this
# dict and use the copy to store gameplay state. This
# way you can replay the game multiple times.

INIT_GAME_STATE = {
    "current_room": game_room,
    "keys_collected": [],
    "target_room": outside
}

game_state = copy.deepcopy(INIT_GAME_STATE)

In [2]:
## Functions

def linebreak():
    """
    Print a line break
    """
    print("\n\n")

def start_game():
    """
    Start the game
    """
    set_difficulty()
    display_time()
    print("You wake up on a couch and find yourself in a strange house with no windows which you have never been to before. You don't remember why you are here and what had happened before. You feel some unknown danger is approaching and you must get out of the house, NOW!")
    play_room(game_state["current_room"])
   
   
def play_room(room):
    """
    Play a room. First check if the room being played is the target room.
    If it is, the game will end with success. Otherwise, let player either
    explore (list all items in this room) or examine an item found here.
    """
    #timer feature
    global t
    game_state["current_room"] = room
    display_time()
    if t == 0:
        print("You couldn't escape from the room. You loose the game!")
        #Total score
        score()
        #new_game()
    elif(game_state["current_room"] == game_state["target_room"]):
        print("Congrats! You escaped the room!")
        # Total score
        score()
        #new_game()        
    else:
        print ("You are now in " + room["name"])
        #timer feature
        display_time()
        intended_action = input("What would you like to do? Type 'explore' or 'examine'?").strip()
        if intended_action == "explore":
            explore_room(room)
            play_room(room)
        elif intended_action == "examine":
            examine_item(input("What would you like to examine?").strip())
        else:
            print("Not sure what you mean. Type 'explore' or 'examine'.")
            play_room(room)
        linebreak()

def explore_room(room):
    """
    Explore a room. List all items belonging to this room.
    """
    items = [i["name"] for i in object_relations[room["name"]]]
    print("You explore the room. This is " + room["name"] + ". You find " + ", ".join(items))

def get_next_room_of_door(door, current_room):
    """
    From object_relations, find the two rooms connected to the given door.
    Return the room that is not the current_room.
    """
    connected_rooms = object_relations[door["name"]]
    for room in connected_rooms:
        if(not current_room == room):
            return room

def examine_item(item_name):
    """
    Examine an item which can be a door or furniture.
    First make sure the intended item belongs to the current room.
    Then check if the item is a door. Tell player if key hasn't been
    collected yet. Otherwise ask player if they want to go to the next
    room. If the item is not a door, then check if it contains keys.
    Collect the key if found and update the game state. At the end,
    play either the current or the next room depending on the game state
    to keep playing.
    """
    current_room = game_state["current_room"]
    next_room = ""
    output = None
   
    for item in object_relations[current_room["name"]]:
        if(item["name"] == item_name):
            output = "You examine " + item_name + ". "
            if(item["type"] == "door"):
                have_key = False
                for key in game_state["keys_collected"]:
                    if(key["target"] == item):
                        have_key = True
                if(have_key):
                    output += "You unlock it with a key you have."
                    next_room = get_next_room_of_door(item, current_room)
                else:
                    output += "It is locked but you don't have the key."
            else:
                if(item["name"] in object_relations and len(object_relations[item["name"]])>0):
                    item_found = object_relations[item["name"]].pop()
                    game_state["keys_collected"].append(item_found)
                    output += "You find a " + item_found["name"] + "."
                else:
                    output += "There isn't anything interesting about it."
            print(output)
            break

    if(output is None):
        print("The item you requested is not found in the current room.")
   
    if(next_room and input("Do you want to go to the next room? Enter 'yes' or 'no'").strip() == 'yes'):
        play_room(next_room)
    else:
        play_room(current_room)
        
# new functions

def countdown():
    global t
    while t > 0:
        time.sleep(1)
        t -= 1
        if game_state["current_room"] == game_state["target_room"]:
            break        

def display_time():
    global t
    mins, secs = divmod(t, 60)
    timeformat = '{:02d}:{:02d}'.format(mins, secs)
    print(timeformat)
    
def set_difficulty():
    global object_relations
    global t
    difficulty = input("What difficulty do you want to play? Type 'easy' or 'normal'.").strip()
    if difficulty.lower() == "easy":
        object_relations = object_relations_easy.copy()
        t = 450
        print("You have selected the 'easy' mode of the game. Hope you enjoy it!")
    elif difficulty.lower() == "normal":
        object_relations = object_relations.copy()
        t = 300
        print("You have selected the 'normal' mode of the game. You are a totally a gamer!")
    else:
        print("Not sure what you mean. Type 'easy' or 'normal'.")

def score():
    global t
    total_score = 0
    if game_state["current_room"] == game_state["target_room"]:
        total_score += 50
    total_score += len(game_state["keys_collected"]) * 15
    total_score += round(t / 5)
    print("Well done!! Your total score is " + str(total_score) + "!!")
    print("Could you beat that?")

# Function to improve that causes problems with the rest of the code

'''
def new_game():
    global game_state
    new_game = input("Would you like to play again? Type 'yes' or 'no'.").strip()
    if new_game.lower() == 'yes':
        game_state = INIT_GAME_STATE.copy()
        print(f'game_state_new {game_state}')
        start_game() 
    elif new_game.lower() == 'no':
        print("Ok, thanks for playing the game!")
    else:
        print("Not sure what you mean. Please, type 'yes' or 'no'.")
        new_game()
'''

'def new_game():\n    global game_state\n    new_game = input("Would you like to play again? Type \'yes\' or \'no\'.").strip()\n    if new_game.lower() == \'yes\':\n        game_state = INIT_GAME_STATE.copy()\n        print(f\'game_state_new {game_state}\')\n        start_game() \n    elif new_game.lower() == \'no\':\n        print("Ok, thanks for playing the game!")\n    else:\n        print("Not sure what you mean. Please, type \'yes\' or \'no\'.")\n        new_game()'

In [ ]:
#create a thread allowing to run simultaniously the timer and the game
countdown_thread = threading.Thread(target = countdown, daemon=True)
countdown_thread.start()

start_game()